### 1. Importing libraries

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
    f1_score,
    precision_score,
    recall_score
)
from utils.utils import split_data
from crack_segmentation.subroutines.loss_metrics import (
    Weighted_Cross_Entropy,
    F1_score_dil,
    F1_score,
    Precision_dil
)

### 2. Loading the dataset

In [2]:
dataset = pd.read_csv('../dataset/dataset_final.csv')

dataset['Filepath'] = dataset['Filepath'].apply(lambda x: '../' + x)

train_df, test_df = train_test_split(
    dataset.sample(frac=1.0, random_state=42),
    train_size=0.80,
    random_state=42
)

preprocess_input = tf.keras.applications.resnet.preprocess_input

train_data, valid_data, test_data = split_data(
        train_df, 
        test_df, 
        image_height=224,
        image_width=224,
        image_channels=1.0, 
        preprocess_input=preprocess_input,
    )

### 3. Importing the UNet-ResNet50 model

In [3]:
model = load_model(
    'crack_segmentation/output/checkpoints/unet_resnet50.h5',
    custom_objects={
        'loss': Weighted_Cross_Entropy(10),
        'F1_score': F1_score,
        'F1_score_dil': F1_score_dil,
        'Precision_dil': Precision_dil
    },
    compile=False
)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)

model.compile(optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

### 4. Evaluating the model

In [4]:
loss, acc = model.evaluate(test_data)

In [5]:
print(f"acc: {acc}")
print(f"loss: {loss}")

### 5. Making predictions

In [6]:
y_pred = np.argmax(model.predict(test_data), axis=-1)

In [7]:
print(y_pred)

In [8]:
metrics = classification_report(test_data.labels, y_pred)

In [9]:
print(metrics)

In [10]:
x = f1_score(test_data.labels, y_pred, average='weighted')
y = precision_score(test_data.labels, y_pred, average='weighted')
z = recall_score(test_data.labels, y_pred, average='weighted')

In [11]:
print(f'F1 score: {x}')
print(f'Precision: {y}')
print(f'Recall: {z}')

In [12]:
cm = confusion_matrix(test_data.labels, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['WITH_CRACK', 'WITHOUT_CRACK'])

disp.plot()